In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
import json
import altair as alt
import re
from sklearn.preprocessing import MinMaxScaler

game_details = pd.read_csv("steam_game_details.csv")
game_stats = pd.read_csv("steam_game_stats.csv")
game_reviews = pd.read_csv("steam_review_data.csv")

game_reviews = game_reviews.dropna()

combined_details = game_details.merge(game_stats, on="app_id")
combined_details['owners_upper'] = combined_details['owners'].apply(
    lambda x: int(re.search(r"\.\. ([\d,]+)", x).group(1).replace(',', '')) if pd.notnull(x) else None
)

combined_details['total'] = combined_details['positive'] + combined_details['negative']

combined_details['positive_ratio'] = combined_details['positive'] / combined_details['total']

scaler = MinMaxScaler()

combined_details['ranked_positive'] = combined_details['positive_ratio'].rank(pct=True)
combined_details['ranked_owners'] = combined_details['owners_upper'].rank(pct=True)

combined_details['popularity_score_ranked'] = (
    0.3 * combined_details['ranked_positive'] +
    0.7 * combined_details['ranked_owners']
)

combined_details['popularity_rating'] = (combined_details['popularity_score_ranked'] * 5).round(1)

In [ ]:
agg_reviews = game_reviews.groupby(by="app_id")['review_text'].apply(lambda x: ' '.join(x))